# A MultiClass Neural Network #

Let's try coding a Multiclass neural network from scratch

In [18]:
# import useful librairies
import numpy as np

In [31]:
# Let's make a multilabel dataset

# Initialize randomness
np.random.seed(42)

# Our dataset is made of coded numbers
zero = np.array([[1],[1],[1],[0],[1],[1],[1]])
one = np.array([[0],[0],[0],[0],[0],[1],[1]])
two = np.array([[0],[1],[1],[1],[1],[1],[0]])
three = np.array([[0],[0],[1],[1],[1],[1],[1]])
four = np.array([[1],[0],[0],[1],[0],[1],[1]])
five = np.array([[1],[0],[1],[1],[1],[0],[1]])
six = np.array([[1],[1],[1],[1],[1],[0],[1]])
seven = np.array([[0],[0],[1],[0],[0],[1],[1]])
eight = np.array([[1],[1],[1],[1],[1],[1],[1]])
nine = np.array([[1],[0],[1],[1],[1],[1],[1]])

# A function to decode our number
def decode_num(x):
    r = x
    r = np.insert(r, 0, [0])
    r = np.insert(r, -2, [0])
    r = r.reshape(9,1)
    r = np.append(r[:3], np.append(r[3:6], r[6:], axis = 1), axis = 1)
    stringa = ''
    stringb = ''
    stringc = ''
    for i,y in enumerate(r):
        for j,v in enumerate(y):
            if i == 0:
                if v == 1 and j == 1:
                    stringa = stringa + '_'
                else:
                    stringa = stringa + ' '
            if i == 1:
                if v == 1 and j == 1:
                    stringb = stringb + '_'
                elif v == 1:
                    stringb = stringb + '|'
                else:
                    stringb = stringb + ' '
            if i == 2:
                if v == 1 and j == 1:
                    stringc = stringc + '_'
                elif v == 1:
                    stringc = stringc + '|'
                else:
                    stringc = stringc + ' '
    print(stringa)
    print(stringb)
    print(stringc)

# Making m examples from our coded numbers
m = 20
# Choosing randomly the numbers for m examples
pre_dataset = np.random.randint(0, 10, (1, m))

# Setting labels
Y = np.zeros(0)
for v in pre_dataset[0]:
    Y = np.append(Y, [0 for x in range(v)] +[1] + [0 for x in range(9-v)])
Y = Y.reshape(m,10)

# Setting inputs
numbers = [zero, one, two, three, four, five, six, seven, eight, nine]
X = np.zeros(0)
for v in pre_dataset[0]:
    X = np.append(X, [numbers[v]])
X = X.reshape(m,7)

# Let's check if everything works find
for i, v in enumerate(Y):
    decode_num(X[i])
    print('label : ' , np.where(v == 1)[0])

 _ 
|_ 
|_|
label :  [6]
 _ 
 _|
 _|
label :  [3]
 _ 
  |
  |
label :  [7]
   
|_|
  |
label :  [4]
 _ 
|_ 
|_|
label :  [6]
 _ 
|_|
 _|
label :  [9]
 _ 
 _|
|_ 
label :  [2]
 _ 
|_ 
|_|
label :  [6]
 _ 
  |
  |
label :  [7]
   
|_|
  |
label :  [4]
 _ 
 _|
 _|
label :  [3]
 _ 
  |
  |
label :  [7]
 _ 
  |
  |
label :  [7]
 _ 
 _|
|_ 
label :  [2]
 _ 
|_ 
 _|
label :  [5]
   
|_|
  |
label :  [4]
   
  |
  |
label :  [1]
 _ 
  |
  |
label :  [7]
 _ 
|_ 
 _|
label :  [5]
   
  |
  |
label :  [1]


In [35]:
# Activation function
def activation(x, function = 'sigmoid'):
    if function == 'sigmoid':
        return 1/(1+np.exp(-x))
    else:
        raise NotImplementedError

In [42]:
# The neural network

# Initialisation
# the architecture (hidden layers)
N = [3]

# Adding the input and output layers
N = [X.shape[1]] + N + [Y.shape[1]]

# Check X, Y and N
print('X shape : ', X.shape)
print('Y shape : ', Y.shape)
print('Archi : ', N)

# Weights and bias
np.random.seed(42)
W = [np.random.randn(N[r], N[r+1]) for r in range(len(N)-2) ]
b = [np.zeros((1, N[r+1])) for r in range(len(N)-2)]

# Hyperparameters
lr = 0.5
epoch = 100

# The training
for i in range(epoch):
    print('LOOP : ', i)
    A = [X]
    # Forward
    for l in range(len(N)-1):
        print('LAYER : ', l+1)
        # the last layer
        if l == len(N) - 2:
            # compute the softmax
            Ypred = np.exp(A[l-1]) / np.sum(np.exp(A[l-1]))
        else:
            
            Z = np.dot(A[l], W[l]) + b[l]
            A.append(activation(Z))
        print(A[l].shape)

X shape :  (20, 7)
Y shape :  (20, 10)
Archi :  [7, 3, 10]
LOOP :  0
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  1
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  2
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  3
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  4
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  5
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  6
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  7
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  8
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  9
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  10
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  11
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  12
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  13
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  14
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  15
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  16
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  17
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  18
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  19
LAYER :  

LAYER :  2
(20, 3)
LOOP :  371
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  372
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  373
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  374
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  375
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  376
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  377
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  378
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  379
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  380
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  381
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  382
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  383
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  384
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  385
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  386
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  387
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  388
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  389
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  390
LAYER :  1
(20, 7)


(20, 7)
LAYER :  2
(20, 3)
LOOP :  746
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  747
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  748
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  749
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  750
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  751
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  752
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  753
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  754
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  755
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  756
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  757
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  758
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  759
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  760
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  761
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  762
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  763
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  764
LAYER :  1
(20, 7)
LAYER :  2
(20, 3)
LOOP :  765
LAYER :  1
